In [1]:
import time
import numpy as np
from tqdm import tqdm

import json
import pandas as pd

from transformers import AutoTokenizer, AutoModel

from utils.eval_utils import micro_precision, micro_recall, f1_score
from utils.openai_utils import LLMTripletExtractor
from utils.verifier_utils import TripletFilter
from utils.structured_dynamic_index_utils_with_db import Aligner

import warnings
import os
import ast

warnings.filterwarnings('ignore')

import re
from unidecode import unidecode

from pymongo.mongo_client import MongoClient
import json

/home/chepurova/Wikontic/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'utils'

In [ ]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [ ]:
import string
def normalize(input_string):
    input_string = unidecode(input_string)
    input_string = input_string.lower()
        # Replace all punctuation with a space
    input_string = re.sub(f"[{re.escape(string.punctuation)}]", " ", input_string)
    
    # Replace multiple spaces with a single space
    input_string = re.sub(r"\s+", " ", input_string)
    
    # Trim leading/trailing whitespace
    return input_string.strip()

In [ ]:
def get_mongo_client(mongo_uri):
    client = MongoClient(mongo_uri)
    return client

mongo_client = get_mongo_client("mongodb://localhost:27018/?directConnection=true")
db = mongo_client.get_database("wikidata_ontology_test")
db.list_collection_names()

['gpt-4o-mini_filtered_triplets',
 'entity_types',
 'gpt-4o-mini_ontology_filtered_triplets',
 'entity_aliases',
 'gpt-4o-mini_triplets',
 'property_aliases',
 'gpt-4o-mini_initial_triplets',
 'properties',
 'entity_type_aliases',
 'gpt-4o-mini_entity_aliases']

In [ ]:
mongo_client.list

In [ ]:
db.get_collection('gpt-4o-mini_triplets').find_one({})

{'_id': ObjectId('68c07ac1d481a3884ea36d1e'),
 'relation': 'instance of',
 'sample_id': '2hop__121145_561444',
 'subject': 'The Real L Word',
 'object_type': 'genre',
 'object': 'reality television series',
 'subject_type': 'television series',
 'completion_token_num': 14,
 'prompt_token_num': 444,
 'qualifiers': [],
 'source_text_id': 0}

In [28]:
# db.get_collection('triplets').delete_many({})

In [29]:
from utils.structured_dynamic_index_utils_with_db import Aligner
from utils.structured_inference_with_db import StructuredInferenceWithDB
model_name = 'gpt-4o-mini'
aligner = Aligner(db, collection_prefix=model_name)
extractor = LLMTripletExtractor(model=model_name)
inferer = StructuredInferenceWithDB(extractor, aligner, db)

with open("musique_200_test.json", "r") as f:
    ds = json.load(f)


In [30]:
print(aligner.entities_vector_index_name)
print(aligner.entity_aliases_collection_name)
print(aligner.entity_type_vector_index_name)
print(aligner.entity_type_aliases_collection_name)
print(aligner.property_aliases_collection_name)
print(aligner.property_collection_name)
print(aligner.entity_type_collection_name)
print(aligner.triplets_collection_name)
print(aligner.filtered_triplets_collection_name)
print(aligner.ontology_filtered_triplets_collection_name)
print(aligner.initial_triplets_collection_name)


gpt-4o-mini_entities
gpt-4o-mini_entity_aliases
entity_type_aliases
entity_type_aliases
property_aliases
properties
entity_types
gpt-4o-mini_triplets
gpt-4o-mini_filtered_triplets
gpt-4o-mini_ontology_filtered_triplets
gpt-4o-mini_initial_triplets


In [31]:
list(db.get_collection(aligner.triplets_collection_name).find({"sample_id": "2hop__121145_561444"}, {"_id": 0, "label": 1, "entity_type": 1, "sample_id": 1}))

[{'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121

In [32]:
db.get_collection(aligner.entity_aliases_collection_name).find_one({})

{'_id': ObjectId('68c07ab4c058f0098dce9e42'),
 'label': 'The Real L Word',
 'entity_type': 'television series',
 'alias': 'The Real L Word',
 'sample_id': '2hop__121145_561444',
 'alias_text_embedding': [0.009889713488519192,
  -0.07133278995752335,
  -0.0059313117526471615,
  -0.030011042952537537,
  0.014623121358454227,
  -0.024449745193123817,
  0.037173010408878326,
  0.0037516832817345858,
  -0.09681671857833862,
  -0.015195912681519985,
  0.01731961779296398,
  -0.07978171855211258,
  -0.03222719952464104,
  -0.01421310007572174,
  -0.04445761814713478,
  -0.002834167331457138,
  -0.031816836446523666,
  -0.06295185536146164,
  -0.00504443421959877,
  0.03671497851610184,
  -0.00648869015276432,
  0.03426196053624153,
  -0.04647936299443245,
  -0.03071165643632412,
  -0.05182984843850136,
  0.08383699506521225,
  0.07503727823495865,
  0.035214636474847794,
  -0.03033829666674137,
  -0.06335499882698059,
  0.04653048887848854,
  0.0007498351042158902,
  -0.06709463149309158,
  -

In [33]:
aligner.retrive_similar_entity_names("a", k=5, sample_id="2hop__121145_561444")

[{'entity': 'director', 'entity_type': 'public office'},
 {'entity': 'novel', 'entity_type': 'literary genre'},
 {'entity': 'author', 'entity_type': 'profession'},
 {'entity': 'Arizona', 'entity_type': 'federated state'},
 {'entity': 'Jews', 'entity_type': 'ethnic group'}]

In [34]:
aligner.retrieve_entity_by_type("Derech Mitzvosecha", sample_id="2hop__121145_561444", entity_type="human")

{'Rabbi Menachem Mendel Schneersohn': 'Rabbi Menachem Mendel Schneersohn',
 'Rabbi Dovber Schneuri': 'Rabbi Dovber Schneuri',
 'Rabbi Mordecai M. Kaplan': 'Rabbi Mordecai M. Kaplan',
 'Chaya Mushka Schneersohn': 'Chaya Mushka Schneersohn',
 'Michael Cera': 'Michael Cera',
 'Jack Cohen': 'Jack Cohen',
 'Ilene Chaiken': 'Ilene Chaiken',
 'Satyajit Ray': 'Satyajit Ray',
 'Clark Duke and Michael Cera': 'Clark Duke and Michael Cera',
 'Clark Duke': 'Clark Duke'}

In [35]:
from pymongo.operations import SearchIndexModel

vector_search_index_model = SearchIndexModel(
    definition={
        "mappings": {
            "dynamic": True,
            "fields": {
                "alias_text_embedding": {
                    "dimensions": 768,
                    "similarity": "cosine",
                    "type": "knnVector",
                },
                "entity_type": {
                    "type": "token"
                },
                "sample_id": {
                    "type": "token"
                }
            },
        }
    },
    name=aligner.entities_vector_index_name,
)

db.get_collection(aligner.entity_aliases_collection_name).create_search_index(model=vector_search_index_model)

'gpt-4o-mini_entities'

In [36]:
with open("musique_200_test.json", "r") as f:
    ds = json.load(f)
    
ds = ds['data'][:]

id2sample = {}
for elem in ds:
    id2sample[elem['id']] = elem

In [37]:
# lens = []
# for sample_id in id2sample.keys():
#     lens.append(len(list(db.get_collection(coll_name).find({"sample_id": sample_id}))))
# assert all([item > 0 for item in lens])
# # lens

In [40]:
# sample_id2ans = {}
# ids = list(db.get_collection(aligner.triplets_collection_name).find({}, {"_id": 0, "sample_id": 1}))
# ids = [item["sample_id"] for item in ids]
# ids = set(ids)
for sample_id in tqdm(ids):
    try:
# for sample_id in id2sample.keys():
        question = id2sample[sample_id]['question']
        identified_entities = inferer.identify_relevant_entities_from_question(question=question, sample_id=sample_id)
        print(len(identified_entities))
        if not identified_entities:
            print(f"[DEBUG] No entities identified for sample_id={sample_id}, question={question}")
        # if len(identified_entities) > 0:
        supporting_triplets, ans = inferer.answer_question(question=question, identified_entities=identified_entities, sample_id=sample_id)

        print(sample_id, " | ", question, ' | ', ans, " | ", normalize(ans), " | ", id2sample[sample_id]['answer'], " | ", normalize(id2sample[sample_id]['answer']))
        print(len(supporting_triplets))
        print("-"*100)
        sample_id2ans[sample_id] = ans
    except Exception as e:
        print(f"Error for sample_id={sample_id}, question={question}")
        continue

  0%|          | 0/200 [00:00<?, ?it/s]

['Indian Rebellion of 1857', 'ruler']
4
Chosen relevant entities:  [{'entity': 'Indian Rebellion of 1857', 'entity_type': 'political event'}, {'entity': 'Indian Rebellion of 1857', 'entity_type': 'event'}, {'entity': 'Mughal Emperor', 'entity_type': 'human'}, {'entity': 'Emperor of Hindustan', 'entity_type': 'title'}]


  0%|          | 1/200 [00:04<15:04,  4.54s/it]

2hop__16844_42173  |  When was the ruler that was dissolved after the Indian Rebellion of 1857 established?  |  1858  |  1858  |  1599  |  1599
85
----------------------------------------------------------------------------------------------------
['first home run', 'new stadium', 'American League Wildcard game']
2
Chosen relevant entities:  [{'entity': 'New York Yankees', 'entity_type': 'sports team'}, {'entity': '2017 American League Wild Card Game', 'entity_type': 'event'}]


  1%|          | 2/200 [00:07<11:30,  3.49s/it]

...

2hop__121145_561444  |  Who did the creator of Derech Mitzvosecha follow?  |  Rabbi Menachem Mendel Schneersohn  |  rabbi menachem mendel schneersohn  |  Dovber Schneuri  |  dovber schneuri
16
----------------------------------------------------------------------------------------------------
['Bancroft', 'county']
1
Chosen relevant entities:  [{'entity': 'Hastings County', 'entity_type': 'geographic region'}]


100%|█████████▉| 199/200 [12:07<00:03,  3.19s/it]

2hop__512773_346751  |  Bancroft's county borders what county?  |  ""  |    |  Haliburton County  |  haliburton county
36
----------------------------------------------------------------------------------------------------
['A Day in the Life', 'castle', 'birth city', 'performer']
2
Chosen relevant entities:  [{'entity': 'A Day in the Life', 'entity_type': 'musical work'}, {'entity': 'Stauffenberg castle of Jettingen', 'entity_type': 'geographic location'}]


100%|██████████| 200/200 [12:11<00:00,  3.66s/it]

3hop1__602329_792411_51423  |  What is the name of the castle in the birth city of the performer who recorded A Day in the Life?  |  Casa Loma  |  casa loma  |  Casa Loma  |  casa loma
57
----------------------------------------------------------------------------------------------------


In [41]:
aligner.retrive_similar_entity_names('Derech Mitzvosecha', k=5, sample_id="2hop__121145_561444")

[{'entity': 'Derech Mitzvosecha', 'entity_type': 'literary work'},
 {'entity': 'Torah im Derech Eretz', 'entity_type': 'concept'},
 {'entity': 'Kehillat Mevakshei Derech', 'entity_type': 'organization'},
 {'entity': 'rabbi', 'entity_type': 'profession'},
 {'entity': 'Sefer Hamitzvos', 'entity_type': 'literary work'}]

In [42]:
list(db.get_collection(aligner.triplets_collection_name).find({"sample_id": "2hop__121145_561444", "subject": "Rabbi Menachem Mendel Schneersohn", 'subject_type': 'literary work'}, {"_id": 0}))

[]

In [43]:
sample_ids = set()
for doc in db.get_collection(aligner.triplets_collection_name).find({}, {"_id": 0, "sample_id": 1}):
    sample_ids.add(doc["sample_id"])
len(sample_ids)

200

In [44]:
db.get_collection(aligner.entity_aliases_collection_name).find_one({"sample_id": "2hop__365236_19320"})

{'_id': ObjectId('68c1ee3ec058f0098dcf2b3c'),
 'label': 'Harvard',
 'entity_type': 'university',
 'alias': 'Harvard',
 'sample_id': '2hop__365236_19320',
 'alias_text_embedding': [0.04436160996556282,
  0.015104020945727825,
  0.0010068441042676568,
  -0.0825362503528595,
  -0.01353803277015686,
  0.0003280105593148619,
  0.007344139274209738,
  0.015294904820621014,
  0.01312243565917015,
  -0.06016584113240242,
  0.006141675170511007,
  -0.0068809338845312595,
  -0.050348397344350815,
  -0.04428769275546074,
  -0.012345030903816223,
  -0.02315518818795681,
  -0.022969013080000877,
  -0.0586497038602829,
  -0.03450579568743706,
  0.02175183594226837,
  -0.016816848888993263,
  0.004256978631019592,
  0.009577223099768162,
  -0.07135123759508133,
  0.03629777207970619,
  -0.03210164234042168,
  -0.006687880959361792,
  0.018349772319197655,
  0.0156754981726408,
  0.04258941486477852,
  0.009269644506275654,
  0.0005786602268926799,
  -0.06181511655449867,
  -0.010824251919984818,
  0.

In [45]:
sample_id2ans

{'2hop__16844_42173': '1858',
 '3hop1__79462_91850_685675': 'New York Yankees',
 '3hop2__92991_89854_76291': 'January 3, 2017',
 '4hop3__673447_132409_145082_35031': '3rd',
 '3hop1__579562_629431_124169': '1952',
 '2hop__622308_61845': 'Frank Lampard',
 '2hop__82713_84616': 'Tony Award for Best Performance by a Leading Actress in a Play',
 '3hop1__159803_89752_75165': '919,628',
 '3hop1__241085_42197_18397': 'Mao Zedong',
 '3hop1__797496_831637_91775': 'Sea, Air, and Land',
 '3hop1__379315_160713_77246': 'Islam',
 '2hop__3739_13529': 'Diego Maradona',
 '4hop1__88342_49853_128008_46748': '""',
 '2hop__839126_472799': 'Eurovision Song Contest 2015',
 '3hop1__617062_127905_87812': "''",
 '4hop3__316459_41402_145282_13584': 'Barcelona represents Catalonia, while Real Madrid represents Castile.',
 '3hop1__19768_19788_15107': "''",
 '2hop__51113_84616': 'Tony Award for Best Performance by a Leading Actress in a Play',
 '3hop1__409517_547811_41132': "''",
 '2hop__14092_8311': '17 May 1220',
 

In [47]:
f1s = []
ems = []
# for sample_id in id2sample.keys():
for sample_id in ids:
    if sample_id in sample_id2ans:
        question = id2sample[sample_id]['question']
        ans = sample_id2ans[sample_id]
        max_f1 = 0
        max_em = 0
        max_f1_entity = ''
        max_em_entity = ''
        gold_answers_variations = [id2sample[sample_id]['answer']]
        gold_answers_variations.extend(id2sample[sample_id]['answer_aliases'])
        for golden_answer in gold_answers_variations:
            golden_answer = normalize(golden_answer)
            ans = normalize(ans)
            f1 = compute_f1(prediction=ans, truth=golden_answer)
            em = golden_answer == ans


            if f1 > max_f1:
                max_f1_entity = golden_answer
            if em > max_em:
                max_em_entity = golden_answer
            max_f1 = max(max_f1, f1)
            max_em = max(max_em, em)
        
        print(sample_id, " | ", ans, " | ", max_em_entity, " | ", max_f1_entity)
        f1s.append(max_f1)
        ems.append(max_em)

sum(ems) / len(ems), sum(f1s)/len(f1s)

2hop__16844_42173  |  1858  |    |  
3hop1__79462_91850_685675  |  new york yankees  |  new york yankees  |  new york yankees
3hop2__92991_89854_76291  |  january 3 2017  |    |  january 2015
4hop3__673447_132409_145082_35031  |  3rd  |    |  
3hop1__579562_629431_124169  |  1952  |  1952  |  1952
2hop__622308_61845  |  frank lampard  |    |  
2hop__82713_84616  |  tony award for best performance by a leading actress in a play  |    |  for best performance by a leading actress in a play in ondine
3hop1__159803_89752_75165  |  919 628  |    |  
3hop1__241085_42197_18397  |  mao zedong  |    |  
3hop1__797496_831637_91775  |  sea air and land  |  sea air and land  |  sea air and land
3hop1__379315_160713_77246  |  islam  |    |  
2hop__3739_13529  |  diego maradona  |    |  
4hop1__88342_49853_128008_46748  |    |    |  
2hop__839126_472799  |  eurovision song contest 2015  |    |  eurovision song contest 1991
3hop1__617062_127905_87812  |    |    |  
4hop3__316459_41402_145282_13584  | 

(0.2814070351758794, 0.37236871169031976)

In [30]:
with open("hotpotqa200.json", "r") as f:
    ds = json.load(f)

ds = ds[:]
id2sample = {}
for elem in ds:
    id2sample[elem['_id']] = elem

sampled_ids = list(id2sample.keys())[:]

FileNotFoundError: [Errno 2] No such file or directory: 'hotpotqa200.json'

In [27]:
sample_id2ans = {}
lens = []
for sample_id in id2sample.keys():
    
    lens.append(len(list(db.get_collection('triplets').find({"sample_id": sample_id}))))
    question = id2sample[sample_id]['question']
    entities = extractor.extract_entities_from_question(question)
    identified_entities = []
    chosen_entities = []
    print(entities)
    if isinstance(entities, dict):
        entities = [entities]

    for ent in entities:
        similar_entities = aligner.retrive_similar_entity_names(entity_name=ent, k=5, sample_id=sample_id)
        exact_entity_match = [e for e in similar_entities if e['entity']==ent]
        if len(exact_entity_match) > 0:
            chosen_entities.extend(exact_entity_match)
        else:
            identified_entities.extend(similar_entities)
            
    chosen_entities.extend(extractor.identify_relevant_entities(question=question, entity_list=identified_entities))
    print("Chosen relevant entities: ", chosen_entities)
    entity_set = {(e['entity'], e['entity_type']) for e in chosen_entities}

    entities4search = list(entity_set)
    or_conditions = []

    for val, typ in entities4search:
        or_conditions.append({
            '$and': [
                {'subject': val},
                {'subject_type': typ}
            ]
        })
        or_conditions.append({
            '$and': [
                {'object': val},
                {'object_type': typ}
            ]
        })

    pipeline = [
        {
            '$match': {
                'sample_id': sample_id,
                '$or': or_conditions
            }
        }
    ]
    entities4search = [ent[0] for ent in entity_set]

    for i in range(5):
        
        or_conditions = []

        for ent in entities4search:
            or_conditions.append({
                '$and': [
                    {'subject': ent},
                ]
            })
            or_conditions.append({
                '$and': [
                    {'object': ent},
                ]
            })

        pipeline = [
            {
                '$match': {
                    'sample_id': sample_id,
                    '$or': or_conditions
                }
            }
        ]

        results = list(db.get_collection('triplets').aggregate(pipeline))

        for doc in results:
            entities4search.append(doc['subject'])
            entities4search.append(doc['object'])

            for q in doc['qualifiers']:
                entities4search.append(q['object'])

        entities4search = list(set(entities4search))
                        
    print(len(results))
    supporting_triplets = []
    for item in results:
        supporting_triplets.append({"subject": item['subject'], 'relation': item['relation'], 'object': item['object'], 'qualifiers': item['qualifiers']})
    

    ans = extractor.answer_question(question=question, triplets=supporting_triplets)
    print(question, ' | ', ans, " | ", id2sample[sample_id]['answer'])
    sample_id2ans[sample_id] = ans

['Derech Mitzvosecha', 'creator of Derech Mitzvosecha']
Chosen relevant entities:  [{'entity': 'second Rebbe of the Chabad Hasidic movement', 'entity_type': 'title'}]
12
Who did the creator of Derech Mitzvosecha follow?  |  third Rebbe of the Chabad Hasidic movement  |  Dovber Schneuri
['NBA scoring title', 'team', 'winner']
Chosen relevant entities:  [{'entity': 'NBA scoring title', 'entity_type': 'award'}, {'entity': 'NBA scoring title', 'entity_type': 'award'}, {'entity': 'NBA scoring title', 'entity_type': 'sports award'}, {'entity': 'Houston Rockets', 'entity_type': 'sports team'}]
212
What team drafted the winner of the NBA scoring title this year?  |  Miami Heat  |  Oklahoma City Thunder
['The Beach', 'Pao Sarasin']
Chosen relevant entities:  [{'entity': 'The Beach', 'entity_type': 'film'}, {'entity': 'The Beach', 'entity_type': 'creative work'}, {'entity': 'Pao Sarasin', 'entity_type': 'human'}, {'entity': 'The Beach', 'entity_type': 'Film'}, {'entity': 'Pao Sarasin', 'entity_t

KeyboardInterrupt: 

In [22]:
f1s = []
ems = []
for sample_id in id2sample.keys():
        question = id2sample[sample_id]['question']
        ans = sample_id2ans[sample_id]
        max_f1 = 0
        max_em = 0
        max_f1_entity = ''
        max_em_entity = ''
        gold_answers_variations = [id2sample[sample_id]['answer']]
        # gold_answers_variations.extend(id2sample[sample_id]['answer_aliases'])
        for golden_answer in gold_answers_variations:
            golden_answer = normalize(golden_answer)
            ans = normalize(ans)
            f1 = compute_f1(prediction=ans, truth=golden_answer)
            em = golden_answer == ans


            if f1 > max_f1:
                max_f1_entity = golden_answer
            if em > max_em:
                max_em_entity = golden_answer
            max_f1 = max(max_f1, f1)
            max_em = max(max_em, em)
        
        print(sample_id, " | ", ans, " | ", max_em_entity, " | ", max_f1_entity)
        f1s.append(max_f1)
        ems.append(max_em)

sum(ems) / len(ems), sum(f1s)/len(f1s)

5a7613c15542994ccc9186bf  |  viacom international media networks europe  |    |  
5adf2fa35542993344016c11  |  jonny craig  |  jonny craig  |  jonny craig
5adfdef9554299025d62a36b  |  bath maine  |  bath maine  |  bath maine
5a7180205542994082a3e856  |  creature comforts  |  creature comforts  |  creature comforts
5a78bc6b554299148911f979  |  lifestyle magazine magazines focused on women interest  |    |  fortnightly women interest magazine
5abdd0f15542991f6610604d  |  failed coup attempt  |    |  a failed coup attempt
5a8e27d45542995a26add46a  |  2009  |    |  
5a881d2355429938390d3eeb  |  love and theft  |  love and theft  |  love and theft
5ae4a1ef55429970de88d9e7  |  2 march 1972  |  2 march 1972  |  2 march 1972
5ae6f2a7554299572ea5464a  |  romeo montague  |    |  
5a845d735542996488c2e52e  |  virginia  |  virginia  |  virginia
5adf5daf5542995534e8c79d  |  no only darren benjamin shepherd is american remi lange is french  |    |  no
5a7c49dc55429935c91b514f  |  science  |    |  
5

(0.48, 0.6642524493050809)

In [23]:
(0.48, 0.6509191159717475)
(0.48, 0.6520100250626567)
(0.48, 0.6642524493050809)

(0.48, 0.6520100250626567)

In [24]:
np.mean([0.6509191159717475, 0.6520100250626567, 0.6642524493050809])

0.6557271967798283

In [25]:
np.std([0.6509191159717475, 0.6520100250626567, 0.6642524493050809])

0.006044692913382812